In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def calculate_portfolio_metrics(prices_df, window=20):
    # Calculate returns
    returns_df = prices_df.pct_change()

    # Calculate rolling returns for the holding period
    rolling_returns = (prices_df.shift(-window) - prices_df) / prices_df

    # Calculate expected returns (mean of rolling returns)
    expected_returns = rolling_returns.mean()

    # Calculate covariance matrix using daily returns
    cov_matrix = returns_df.cov() * window

    return expected_returns, cov_matrix

def portfolio_performance(weights, expected_returns, cov_matrix):
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_risk

def optimize_portfolio(expected_returns, cov_matrix, target_risk=None, target_return=None):
    num_assets = len(expected_returns)

    def portfolio_volatility(weights):
        return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    def portfolio_return(weights):
        return np.sum(expected_returns * weights)

    # Constraints
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # weights sum to 1

    if target_risk is not None:
        constraints.append({
            'type': 'eq',
            'fun': lambda x: portfolio_volatility(x) - target_risk
        })

    if target_return is not None:
        constraints.append({
            'type': 'eq',
            'fun': lambda x: portfolio_return(x) - target_return
        })

    # Bounds (0 to 1 for long-only portfolio)
    bounds = tuple((0, 1) for _ in range(num_assets))

    # Initial guess (equal weights)
    initial_weights = np.array([1/num_assets] * num_assets)

    # Objective function
    if target_risk is None and target_return is None:
        # Maximize Sharpe Ratio
        risk_free_rate = 0.03/252  # Adjust as needed
        def objective(weights):
            port_return, port_risk = portfolio_performance(weights, expected_returns, cov_matrix)
            sharpe = (port_return - risk_free_rate) / port_risk
            return -sharpe
    elif target_risk is not None:
        # Maximize return for given risk
        objective = lambda x: -portfolio_return(x)
    else:
        # Minimize risk for given return
        objective = portfolio_volatility

    # Optimize
    result = minimize(objective, initial_weights, method='SLSQP',
                     bounds=bounds, constraints=constraints)

    return result.x

def generate_efficient_frontier(expected_returns, cov_matrix, points=100):
    # Find minimum volatility portfolio
    min_vol_weights = optimize_portfolio(expected_returns, cov_matrix)
    min_vol_return, min_vol_risk = portfolio_performance(min_vol_weights, expected_returns, cov_matrix)

    # Find maximum return portfolio (invest all in highest return asset)
    max_return_idx = np.argmax(expected_returns)
    max_return = expected_returns[max_return_idx]
    max_weights = np.zeros(len(expected_returns))
    max_weights[max_return_idx] = 1
    _, max_risk = portfolio_performance(max_weights, expected_returns, cov_matrix)

    # Generate efficient frontier points
    target_returns = np.linspace(min_vol_return, max_return, points)
    efficient_frontier = []

    for target_return in target_returns:
        weights = optimize_portfolio(expected_returns, cov_matrix, target_return=target_return)
        return_, risk = portfolio_performance(weights, expected_returns, cov_matrix)
        efficient_frontier.append((risk, return_))

    return np.array(efficient_frontier)

def get_optimal_portfolio(expected_returns, cov_matrix, risk_tolerance):
    """
    risk_tolerance: 0 to 1, where 0 is most conservative (minimum volatility)
    and 1 is most aggressive (maximum return)
    """
    efficient_frontier = generate_efficient_frontier(expected_returns, cov_matrix)
    risks = efficient_frontier[:, 0]

    # Find target risk based on risk tolerance
    min_risk = np.min(risks)
    max_risk = np.max(risks)
    target_risk = min_risk + risk_tolerance * (max_risk - min_risk)

    # Get optimal portfolio for target risk
    optimal_weights = optimize_portfolio(expected_returns, cov_matrix, target_risk=target_risk)
    return optimal_weights

def plot_efficient_frontier(efficient_frontier, current_portfolio=None):
    plt.figure(figsize=(10, 6))
    plt.plot(efficient_frontier[:, 0], efficient_frontier[:, 1], 'b-', label='Efficient Frontier')

    if current_portfolio is not None:
        plt.plot(current_portfolio[0], current_portfolio[1], 'r*', markersize=15, label='Selected Portfolio')

    plt.xlabel('Risk (Standard Deviation)')
    plt.ylabel('Expected Return')
    plt.title('Efficient Frontier')
    plt.legend()
    plt.grid(True)
    plt.show()